In [1]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sail.transformers.river.preprocessing import StandardScaler
from sail.models.torch.rnn import RNNRegressor

#### Load Data


In [2]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
learner_gru = RNNRegressor(
    input_units=12,
    output_units=1,
    hidden_units=100,
    n_hidden_layers=3,
    lr=0.001,
    cell_type="GRU",
    verbose=0,
)

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", learner_gru),
]
sail_pipeline = SAILPipeline(
    steps=steps, scoring=["R2"], verbosity_level=1, verbosity_interval=100
)

#### Train Test Split


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

#### Start Incremental Training


In [6]:
y_preds = []
y_true = []
batch_size = 10

for start in range(0, 5000, batch_size):
    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_pipeline.partial_fit(X, y)


    
>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.031s, Batch Size=10, P_Score=0]            

    
>> Epoch: 100 | Samples Seen: 990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=-.265]            

    
>> Epoch: 200 | Samples Seen: 1990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.204]            

    
>> Epoch: 300 | Samples Seen: 2990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.005s, Batch Size=10, P_Sco

#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

[2023-12-12 17:48:58:666] - INFO - SAIL (PyTorch) - Model saved successfully.


'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

SAIL Pipeline Progressive Score: 100%========== [Points: 4990/4990, Elapsed:0.0108s, Metric=R2, Batch Size=4990, P_Score=0.511]


In [9]:
y_preds = []
y_true = []
batch_size = 10

for start in range(5001, 10768, batch_size):
    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
    501       71.8888  0.0018
    502      566.5175  0.0015
    503      131.7938  0.0012
    504      114.2689  0.0013
    505      248.9749  0.0013
    506      165.8501  0.0012
    507      131.9454  0.0014
    508      439.9035  0.0012
    509      119.8833  0.0012
    510      183.1458  0.0014
    511      341.1264  0.0011
    512      357.0931  0.0010
    513      175.2396  0.0011
    514      166.6846  0.0012
    515      185.7204  0.0010
    516      311.2947  0.0011
    517      263.8472  0.0012
    518      263.6564  0.0010
    519      250.8266  0.0011
    520      263.2386  0.0010
    521      611.0502  0.0012
    522      255.3256  0.0011
    523      201.8802  0.0013
    524      402.3539  0.0020
    525      145.4151  0.0011
    526      472.4382  0.0016
    527      230.3152  0.0013
    528       95.9053  0.0012
    529      201.5382  0.0013
    530       40.2959  0.0011
    531      150.1678  0.0010
    532   


    
>> Epoch: 600 | Samples Seen: 5990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

    600      440.3103  0.0017


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.548]            

    601      349.0640  0.0040
    602      148.2638  0.0014
    603      418.7603  0.0011
    604      224.7227  0.0015
    605      493.5359  0.0010
    606      148.5124  0.0010
    607      443.9808  0.0013
    608      376.0975  0.0010
    609      286.0743  0.0011
    610       87.8358  0.0012
    611      458.2039  0.0012
    612      509.4558  0.0010
    613       55.7207  0.0011
    614      274.4852  0.0010
    615      374.0755  0.0010
    616      346.5542  0.0012
    617      791.0632  0.0012
    618      153.6690  0.0013
    619      272.5239  0.0010
    620      640.4315  0.0010
    621      360.7834  0.0010
    622      255.1215  0.0010
    623      287.2895  0.0010
    624      110.7721  0.0010
    625      228.7529  0.0010
    626      309.4091  0.0010
    627      585.6977  0.0010
    628      260.3463  0.0010
    629       75.5383  0.0013
    630       80.2034  0.0011
    631      264.2268  0.0010
    632      267.0144  0.0010
    633      461.2321  0.0011
    634   

...


    
>> Epoch: 1000 | Samples Seen: 9990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

   1000      401.9387  0.0020


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.621]            

   1001      315.9359  0.0013
   1002      216.0316  0.0010
   1003      361.9715  0.0011
   1004      156.3135  0.0016
   1005       37.7104  0.0035
   1006      125.2292  0.0011
   1007      286.9528  0.0012
   1008      119.6226  0.0012
   1009      143.3138  0.0011
   1010      663.3904  0.0011
   1011      180.2863  0.0012
   1012      184.1097  0.0011
   1013      112.3237  0.0010
   1014      106.8147  0.0010
   1015      614.5563  0.0011
   1016      107.2000  0.0010
   1017      315.1335  0.0010
   1018      255.0704  0.0011
   1019      174.6810  0.0012
   1020      312.7851  0.0010
   1021      227.2909  0.0010
   1022      300.5590  0.0013
   1023      231.7662  0.0012
   1024      192.6102  0.0011
   1025      252.9846  0.0010
   1026       69.8974  0.0013
   1027      194.1710  0.0011
   1028      611.2283  0.0010
   1029      309.7976  0.0013
   1030      198.2696  0.0010
   1031       77.0998  0.0011
   1032      689.9325  0.0011
   1033      324.5117  0.0015
   1034   

   1040      468.0385  0.0016
   1041      217.8708  0.0011
   1042      115.3591  0.0010
   1043      142.9978  0.0011
   1044      512.0624  0.0011
   1045      260.4395  0.0011
   1046       37.6605  0.0010
   1047      392.6596  0.0015
   1048       22.9526  0.0011
   1049      155.6678  0.0011
   1050      129.3821  0.0011
   1051      181.1139  0.0011
   1052      138.3929  0.0012
   1053      286.3607  0.0011
   1054      353.3319  0.0012
   1055      112.5617  0.0015
   1056      231.1056  0.0011
   1057      265.6822  0.0010
   1058      282.5490  0.0012
   1059      152.9145  0.0013
   1060      493.5140  0.0010
   1061      155.3964  0.0013
   1062      213.4594  0.0011
   1063       83.6210  0.0011
   1064      157.3374  0.0010
   1065      245.5015  0.0011
   1066      302.1479  0.0010
   1067      372.0648  0.0014
   1068      315.1951  0.0011
   1069      443.5312  0.0012
   1070      384.0384  0.0011
   1071      265.9665  0.0012
   1072      205.3786  0.0011
   1073   

#### Make Prediction on Hold out set


In [10]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):
    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [11]:
sail_new_pipeline.get_progressive_score

0.6283018631213579

In [12]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title="")
fig.show()